In [ ]:
import getpass
import pprint
import os

from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"): 
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

from openai import OpenAI

In [32]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader, Docx2txtLoader, DirectoryLoader, UnstructuredWordDocumentLoader, UnstructuredExcelLoader, CSVLoader
from langchain.text_splitter import CharacterTextSplitter, SpacyTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain_iris import IRISVector

In [3]:
client = OpenAI()
embeddings = OpenAIEmbeddings()

/var/folders/0n/6bwdvxdj6tz_d082n8rflq140000gn/T/ipykernel_41609/1207519732.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
# Under the hood, this becomes a SQL table. CANNOT have '.' in the name

In [ ]:
# loader = DirectoryLoader('data', glob='*.docx', loader_cls=Docx2txtLoader)
# docs = loader.load()
# len(docs)

11

In [ ]:
# text_splitter = SpacyTextSplitter(chunk_size=400, chunk_overlap=20)
# docs = text_splitter.split_documents(docs)

/Users/admintech_solomonsoh/.pyenv/versions/3.10/envs/iris-env/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
# # This creates a persistent vector store (a SQL table). You should run this ONCE only
# db = IRISVector.from_documents(
#     embedding=embeddings,
#     documents=docs,
#     collection_name=COLLECTION_NAME,
#     connection_string=CONNECTION_STRING,
# )

In [ ]:
COLLECTION_NAME = "cancer_db"
# Subsequent calls to reconnect to the database and make searches should use this.  
db = IRISVector(
    embedding_function=embeddings,
    dimension=1536,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [14]:
print(f"Number of docs in vector store: {len(db.get()['ids'])}")

Number of docs in vector store: 128


In [36]:
f = open("data/s_test.txt", "r", encoding='ISO-8859-1')
# query = "new technology"
scenario = f.read()

f = open("data/knowledge.docx", "r", encoding='ISO-8859-1')
# query = "new technology"
knowledge = f.read()

In [16]:
docs_with_score = db.similarity_search_with_score(scenario, 2)
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.057097903683261
Premise: Pt has a big tumor but cannot accept mastectomy, discuss neoadjuvant chemotherapy- risks and benefits and agreeable for trial of nact before mastectomy

Patient Scenario

Dr. Tan: Good morning, Ms. Chua.

Thank you for coming in today.

I understand that you’ve been thinking a lot about your treatment options.

How are you feeling?



Ms. Chua: Good morning, Dr. Tan.

I’m very worried.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.063451157120054
NACT can reduce the tumor size, and if the response is good, we may be able to proceed with a lumpectomy instead of a mastectomy.

However, the response to chemotherapy varies between patients.

If the tumor does not shrink enough, we may still need to do a mastectomy.



Ms. Chua: I see.

What are the ris

In [17]:
embedding_vector = embeddings.embed_query(scenario)
res = db.similarity_search_by_vector(embedding_vector)
res

[Document(metadata={'source': 'data/s5.docx'}, page_content='Premise: Pt has a big tumor but cannot accept mastectomy, discuss neoadjuvant chemotherapy- risks and benefits and agreeable for trial of nact before mastectomy\n\nPatient Scenario\n\nDr. Tan: Good morning, Ms. Chua.\n\nThank you for coming in today.\n\nI understand that you’ve been thinking a lot about your treatment options.\n\nHow are you feeling?\n\n\n\nMs. Chua: Good morning, Dr. Tan.\n\nI’m very worried.'),
 Document(metadata={'source': 'data/s5.docx'}, page_content='NACT can reduce the tumor size, and if the response is good, we may be able to proceed with a lumpectomy instead of a mastectomy.\n\nHowever, the response to chemotherapy varies between patients.\n\nIf the tumor does not shrink enough, we may still need to do a mastectomy.\n\n\n\nMs. Chua: I see.\n\nWhat are the risks of trying chemotherapy first?'),
 Document(metadata={'source': 'data/s5.docx'}, page_content='If there’s even a chance I can avoid mastectomy

In [18]:
full_res = ''
for each_res in res:
    full_res = full_res + '\n\n' +each_res.page_content

In [19]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": 
                f"""
                A medical doctor with domain knowledge in breast cancer after having trained with a wealth of knowledge in these topics: {knowledge}.
                Augment your data with results from {full_res}
                """
        },
        {
            "role": "user",
            "content": 
                f"""
                Given patient's consultation with the doctor in this {scenario}, 
                recommend 
                1. the best course of treatment
                2. provide justifications for the course
                3. provide chain of thought to reach those justifications
                4. highlight risks to patient
                """
        }
    ]
)


In [22]:
pprint.pp(completion.choices[0].message.content)

('**1. Best Course of Treatment:**\n'
 '\n'
 'The recommended course of treatment for Ms. Chua is to proceed with '
 'neoadjuvant chemotherapy (NACT) prior to surgery.\n'
 '\n'
 '**2. Justifications for the Course:**\n'
 '\n'
 '- **Tumor Reduction Potential**: Neoadjuvant chemotherapy has the primary '
 'goal of reducing the size of the tumor, making it more amenable to '
 'breast-conserving surgery (lumpectomy) instead of a mastectomy.\n'
 '  \n'
 '- **Patient Preference**: Ms. Chua has expressed a strong desire to avoid '
 'mastectomy if possible. NACT aligns with her preferences while also '
 'providing a medically sound approach.\n'
 '  \n'
 '- **Assessment and Monitoring**: NACT allows for close monitoring of the '
 "tumor's response through imaging studies. If the tumor shrinks "
 'significantly, a more conservative surgical option can be pursued, which is '
 "important for Ms. Chua's psychological well-being and quality of life.\n"
 '\n'
 '- **No Immediate Surgical Risks**: By o

In [43]:
loader = CSVLoader('data/treatment_selection.csv')#, csv_args={'fieldnames':['']})
docs = loader.load()
len(docs)

12

In [44]:
COLLECTION_NAME = "pictures_db"
# This creates a persistent vector store (a SQL table). You should run this ONCE only
db1 = IRISVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [46]:
docs_with_score = db1.similarity_search_with_score(completion.choices[0].message.content, 1)
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.181232394924638
﻿surgery_type: mastectomy
benefit: reduced local recurrence risk; rarely requires radiotherapy
consideration: permanent breast removal; needs drain management; longer recovery time
tag: img9
--------------------------------------------------------------------------------


In [57]:
image_chosen = doc.page_content.split('\n')[-1].split(': ')[-1] + ".jpeg"

In [58]:
image_chosen

'img9.jpeg'

# Misc

In [ ]:
# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {
#             "role": "system", 
#             "content": """
#             Factors For Lumpectomy:
# Breast Conservation – Preserves the natural breast, which may be important for some women, though Mdm Ang has indicated she may be able to accept a flat chest.
# Less Invasive Surgery – Typically, a shorter recovery time compared to a mastectomy.
# Factors Against Lumpectomy:
# Need for Frequent Follow-Ups – Requires post-surgical radiotherapy, which means multiple hospital visits, a significant concern for Mdm Ang since she finds it difficult to travel to the hospital frequently.
# Risk of Second Surgery – If the lumpectomy does not achieve clear margins, a second surgery may be required, which Mdm Ang wants to avoid.
# Overall Treatment Burden – The combination of surgery and radiotherapy means a longer treatment course, which may not be ideal given her preference for a one-time treatment.
# Since Mdm Ang prioritizes minimizing hospital visits and avoiding the possibility of a second surgery, mastectomy without reconstruction aligns better with her needs.
# """},
#         {
#             "role": "user",
#             "content": f"""
# 1) cannot accept mastectomy
# 2) wants reconstruction but cannot accept implant - can accept tram or LD flap 
# 3) only wants lumpectomy ok for second surgery
# 4) tumor too big but really only want breast conserving and considering oncoplastic surgery - accepting of a slightly longer scar to maintain symmetry of best 
# 5) big tumor but cannot accept mastectomy, discuss neoadjuvant chemotherapy- risks and benefits and agreeable for trial of nact before mastectomy
# 6) cost concerns. 
# Deciding between Breast conserving but with radiotherapy versus mastectomy without recon 
# Recon too expensive
# 7) concern about drain management and no caregiver - prefers fast recovery - lumpectomy 
# 8 ) doesn’t want radiotherapy strongly  - mastectomy the.
#             ."""
#         }
#     ]
# )

In [ ]:
# from langchain.text_splitter import CharacterTextSplitter

# text = "Your long document text here..."

# splitter = CharacterTextSplitter(
#     separator="\n\n",
#     chunk_size=1000,
#     chunk_overlap=200
# )

# chunks = splitter.split_text(text) #you can also split documents using split_documents

# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text = "Your long document text here..."

# splitter = RecursiveCharacterTextSplitter(
#     separators=["\n\n", "\n", " ", ""],
#     chunk_size=1000,
#     chunk_overlap=200,
#     length_function=len
# )

# chunks = splitter.split_text(text)